In [1]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [2]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\MI\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary'])
]


In [3]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\MI\20161108__mi__general__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\MI\20161108__mi__special__general__precinct.csv


In [4]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\MI\20160308__mi__primary__president__precinct.csv


In [5]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]       
            df = df[~df['precinct'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)] 

        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]     
        
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)


# Checking any suspicious precinct
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]


In [6]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

,precinct,party,candidate,votes
0,ALCONAALCONA TOWNSHIP 1,REP,Jeb Bush,3
1,ALCONACALEDONIA TOWNSHIP 1,REP,Jeb Bush,0
2,ALCONACURTIS TOWNSHIP 1,REP,Jeb Bush,3
3,ALCONAGREENBUSH TOWNSHIP 1,REP,Jeb Bush,1
4,ALCONAGUSTIN TOWNSHIP 1,REP,Jeb Bush,0
...,...,...,...,...
19451,KALAMAZOOPORTAGE CITY 19,DEM,Fuente,0
19452,KALAMAZOOPORTAGE CITY 19,DEM,O'malley,0
19453,KALAMAZOOPORTAGE CITY 19,DEM,Sanders,195
19454,KALAMAZOOPORTAGE CITY 19,DEM,Uncommitted,9


In [7]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
Uncommitted                 1942
Rand Paul                    884
Marco Rubio                  884
Bernie Sanders               884
Martin J. O'Malley           884
Hillary Clinton              884
Donald J. Trump              884
John R. Kasich               884
George Pataki                884
Rick Santorum                884
Jeb Bush                     863
Ben Carson                   863
Mike Huckabee                863
Lindsey Graham               863
Carly Fiorina                863
Ted Cruz                     863
Chris Christie               863
Roque Rocky De La Fuente     520
Write-ins                    465
Rocky De La Fuente           364
Write-In                     119
O'malley                     108
Pataki                       108
Fuente                       108
Clinton                      108
Write-in                     108
Trump                        108
Santorum                     108
Rubio                        108
Paul                         108


In [8]:
# Cleaning Candidates

# Turning all primary data to uppercase
# primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

# unwanted_keywords = [
#     r"\w*VOTE\w*",
#     r"\w*UNCOM\w*",
#     r"\w*TOTAL\w*",
#     r"Blank Votes", r"All Others",r"Total Votes Cast",r"No Preference"
# ]

# pattern = "|".join(unwanted_keywords)

# # Assuming candidate column is already string and uppercase
# primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]
primary_data = primary_data[~primary_data["candidate"].isin(["Uncommitted", "Write-ins","Write-in","Write-In","Ballots Cast","Write-In Votes"])] 
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
# # Fixing the , candidate
# primary_data["candidate"] = (
#     primary_data["candidate"].str.split(",")
#     .str[0]
#     .str.strip()
#     )

# # Fixing De la Fuente
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*Fuen\w*", case=False, na=False),
#     "candidate"
# ] = "LA FUENTE"

# #Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# # Fixing Huckabee
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
#     "candidate"
# ] = "HUCKABEE"

# # Fixing Fiorina
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
#     "candidate"
# ] = "FIORINA"

# # Fixing Gray
# primary_data.loc[
#     primary_data["candidate"].str.contains("GREY", case=False, na=False),
#     "candidate"
# ] = "GRAY"

# # Fixing Wilson
# primary_data.loc[
#     primary_data["candidate"].str.contains("WISON", case=False, na=False),
#     "candidate"
# ] = "WILSON"


# # Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

# primary_data.loc[
#     primary_data["candidate"].str.contains("De La Fuen", case=False, na=False),
#     "candidate"
# ] = "FUENTE D"

# primary_data.loc[:,"candidate"] = (
#     primary_data["candidate"].str.split().str[0].str.upper()
# )

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\986686112.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\986686112.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]


candidate
PATAKI      992
PAUL        992
O'MALLEY    992
FUENTE      992
CLINTON     992
TRUMP       992
SANTORUM    992
RUBIO       992
SANDERS     992
KASICH      992
CARSON      971
HUCKABEE    971
GRAHAM      971
FIORINA     971
CRUZ        971
CHRISTIE    971
BUSH        971
Name: count, dtype: int64

In [9]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
REP    12749
DEM     3968
Name: count, dtype: int64

In [10]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown

#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# primary_data["party"] = primary_data.apply(
#     lambda row: fill_party_from_data(row, primary_data),
#     axis=1
# )

primary_data["party"].value_counts(dropna=False)


party
REP    12749
DEM     3968
Name: count, dtype: int64

In [11]:
# Cleaning Party
# Turning all general data party to uppercase
primary_data["party"] = primary_data["party"].astype(str).str.upper()

primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)
primary_data["party"] = (
    primary_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK"
    })
)
primary_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\1660796865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\1660796865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\1660796865.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

party
REP    12749
DEM     3968
Name: count, dtype: int64

In [12]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\2521670330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_FUENTE,pri_dem_O'MALLEY,pri_dem_SANDERS,pri_rep_BUSH,pri_rep_CARSON,pri_rep_CHRISTIE,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_GRAHAM,pri_rep_HUCKABEE,pri_rep_KASICH,pri_rep_PATAKI,pri_rep_PAUL,pri_rep_RUBIO,pri_rep_SANTORUM,pri_rep_TRUMP
0,ALCONAALCONA TOWNSHIP 1,48,0,0,42,3,4,1,61,2,0,1,44,0,0,18,1,141
1,ALCONACALEDONIA TOWNSHIP 1,53,0,1,34,0,1,1,40,0,0,0,32,1,0,15,0,112
2,ALCONACURTIS TOWNSHIP 1,72,0,0,63,3,5,0,46,2,0,1,24,0,0,11,0,128
3,ALCONAGREENBUSH TOWNSHIP 1,67,0,1,66,1,5,1,39,0,0,0,46,0,2,17,1,121
4,ALCONAGUSTIN TOWNSHIP 1,27,0,0,24,0,0,0,23,0,0,0,18,1,0,5,0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,SCHOOLCRAFTMANISTIQUE CITY 3,25,0,0,28,0,2,0,21,0,0,0,15,1,0,7,0,44
988,SCHOOLCRAFTMANISTIQUE TOWNSHIP 1,31,0,0,41,2,0,0,24,0,0,0,37,0,0,14,0,71
989,SCHOOLCRAFTMUELLER TOWNSHIP 1,12,0,0,12,0,0,0,10,0,0,0,7,0,0,3,0,12
990,SCHOOLCRAFTSENEY TOWNSHIP 1,4,0,0,3,0,0,0,5,0,0,0,5,0,0,2,0,22


In [13]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
            df = df[~df['precinct'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]     

        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]     
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]


In [14]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,ALCONAALCONA TOWNSHIP 1,NPA,Tom Hoefling,0
1,ALCONAALCONA TOWNSHIP 1,NPA,Michael A. Maturen,0
2,ALCONAALCONA TOWNSHIP 1,NPA,Monica Moorehead,0
3,ALCONAALCONA TOWNSHIP 1,NPA,Evan Mcmullin,0
4,ALCONAALCONA TOWNSHIP 1,NPA,Laurence Kotlikoff,0
...,...,...,...,...
65996,WEXFORD9999,DEM,Hillary Clinton,0
65997,WEXFORD9999,NLP,Emidio Mimi Soltysik,0
65998,WEXFORD9999,REP,Donald J. Trump,0
65999,WEXFORD9999,UST,Darrell L. Castle,0


In [15]:
# Viewing candidate data
general_data["candidate"].value_counts(dropna=False)

candidate
Tom Hoefling            5077
Michael A. Maturen      5077
Monica Moorehead        5077
Evan Mcmullin           5077
Laurence Kotlikoff      5077
Ben Hartnell            5077
Cherunda Fox            5077
Jill Stein              5077
Hillary Clinton         5077
Emidio Mimi Soltysik    5077
Donald J. Trump         5077
Darrell L. Castle       5077
Gary Johnson            5077
Name: count, dtype: int64

In [16]:
# Cleaning Candidates

# Turning all general data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


# unwanted_keywords = [
#     r"\w*VOTE\w*",
#     r"\w*UNCOM\w*",
#     r"\w*TOTAL\w*",
#     r"\w*WRITE[\s-]\w*",
#     r"NAN",
#     r"UNCERTIFIED",
#     r"UNVERIFIED",
#     r"NONE OF THE ABOVE",
#     r"LBT",
#     r"DEM",
#     r"REP",
#     r"GRN",
#     r"NAN"

# ]

# pattern = "|".join(unwanted_keywords)

# # FILTER OUT TRASH WORDS
# general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# # Fixing Darrell Castle"
# general_data.loc[
#     general_data["candidate"].str.contains("\w*ASTLE\w*", case=False, na=False),
#     "candidate"
# ] = "CASTLE"

# general_data.loc[
#     general_data["candidate"].str.contains("\w*ATUREN\w*", case=False, na=False),
#     "candidate"
# ] = "MATUREN"

# # Fixing mixed president + vice_president by /
# general_data["candidate"] = (
#     general_data["candidate"].str.split("/")
#     .str[0]
#     .str.strip()
#     )

# #Selecting only last name
# general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# # Fixing the , candidate
# # general_data["candidate"] = (
# #     general_data["candidate"].str.split(",")
# #     .str[0]
# #     .str.strip()
# #     )




# # Fixing Cubbler
# general_data.loc[
#     ( 
#         general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "candidate")
# ] = "CUBBLER"


# # Fixing Kotlikoff
# general_data.loc[
#     general_data["candidate"].str.contains("\w*TIKOFF\w*", case=False, na=False),
#     "candidate"
# ] = "KOTLIKOFF"

# # Fixing Valdivia
# general_data.loc[
#     general_data["candidate"].str.contains("\w*VALDIVA\W*", case=False, na=False),
#     "candidate"
# ] = "VALDIVIA"

# # Fixing HOEFLING
# general_data.loc[
#     general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
#     "candidate"
# ] = "HOEFLING"


# # Fixing McMullin
# general_data.loc[
#     general_data["candidate"].str.contains("MCMULLEN", case=False, na=False) ,
#     "candidate"
# ] = "MCMULLIN"

# Fixing LASTNAME + First name Initial
# general_data.loc[:,"candidate"] = (
#     general_data["candidate"].str.split().str[0].str.upper()
# )


# #Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\4180683526.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\4180683526.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].str.split().str[-1]


candidate
HOEFLING     5077
MATUREN      5077
MOOREHEAD    5077
MCMULLIN     5077
KOTLIKOFF    5077
HARTNELL     5077
FOX          5077
STEIN        5077
CLINTON      5077
SOLTYSIK     5077
TRUMP        5077
CASTLE       5077
JOHNSON      5077
Name: count, dtype: int64

In [17]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
NPA    35539
GRN     5077
DEM     5077
NLP     5077
REP     5077
UST     5077
LIB     5077
Name: count, dtype: int64

In [18]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# general_data["party"] = general_data.apply(
#     lambda row: fill_party_from_data(row, general_data),
#     axis=1
# )

# # Fill remaining party using general master CSV
# master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv") # USE YOUR OWN ADDRESS
# general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


party
NPA    35539
GRN     5077
DEM     5077
NLP     5077
REP     5077
UST     5077
LIB     5077
Name: count, dtype: int64

In [19]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GRN",
        "GREEN AND RAINBOW":"GRN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LIBERTARIN":"LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK",
        "NPA":"IND",
        "UST":"CON",
        "(WRITE-IN)":"IND"
    })
)

general_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\1506123479.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = general_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\1506123479.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = (


party
IND    35539
GRN     5077
DEM     5077
NLP     5077
REP     5077
CON     5077
LIB     5077
Name: count, dtype: int64

In [20]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [21]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27084\1829951435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate_column"] = (


candidate_column,precinct,gen_con_CASTLE,gen_dem_CLINTON,gen_grn_STEIN,gen_ind_FOX,gen_ind_HARTNELL,gen_ind_HOEFLING,gen_ind_KOTLIKOFF,gen_ind_MATUREN,gen_ind_MCMULLIN,gen_ind_MOOREHEAD,gen_lib_JOHNSON,gen_nlp_SOLTYSIK,gen_rep_TRUMP
0,ALCONA9999,0,0,0,0,0,0,0,16,3,0,0,0,0
1,ALCONAALCONA TOWNSHIP 1,4,199,2,0,0,0,0,0,0,0,13,0,500
2,ALCONACALEDONIA TOWNSHIP 1,5,178,4,0,0,0,0,0,0,0,14,0,441
3,ALCONACURTIS TOWNSHIP 1,3,214,13,0,0,0,0,0,0,0,8,0,401
4,ALCONAGREENBUSH TOWNSHIP 1,2,256,4,0,0,0,0,0,0,0,22,0,501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,WEXFORDSELMA TOWNSHIP 1,4,322,14,0,0,0,0,0,0,0,36,1,729
5073,WEXFORDSLAGLE TOWNSHIP 1,2,92,0,0,0,0,0,0,0,0,12,0,179
5074,WEXFORDSOUTH BRANCH TOWNSHIP 1,0,76,3,0,0,0,0,0,0,0,5,0,127
5075,WEXFORDSPRINGVILLE TOWNSHIP 1,0,176,10,0,0,0,0,0,0,0,28,0,494


In [22]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts




In [23]:


combined.to_csv("MI.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("MI_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("MI_general_filtered.csv", index=False)

In [24]:
print(f"primary: {len(primary_precincts)}, general: {len(general_precincts)}, combined: {len(combined)}")

primary: 992, general: 5077, combined: 407


In [25]:
print(f"primary: {len(pri_combined_df['county'].unique())}, general: {len(gen_combined_df['county'].unique())}")


primary: 22, general: 83


In [26]:
pri_combined_df['county'].unique()

array(['Alcona', 'Alger', 'Allegan', 'Schoolcraft', 'Barry', 'Charlevoix',
       'Cheboygan', 'Clare', 'Gogebic', 'Gratiot', 'Houghton', 'Iron',
       'Keweenaw', 'Luce', 'Mackinac', 'Marquette', 'Montcalm',
       'Montmorency', 'Ontonagon', 'Oscoda', 'Oakland', 'Kalamazoo'],
      dtype=object)